In [37]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta
import psycopg2
import database

In [59]:

def not_contains_query(query, title, description):
    """Verifica se a consulta não está no título ou descrição."""
    for word in query.lower().split(' '):
        if word in title.lower() or word in description.lower():
            return False
    print("Não contém a consulta: ", query, title, description)
    return True


def fetch_g1_news(query, num_articles=5):
    """Obtém notícias do site G1 com base em uma consulta."""

    base_url = f'https://g1.globo.com/busca/?q={query.replace(" ", "+")}&order=recent&species=notícias'
    articles = []
    page = 1

    while len(articles) < num_articles:
        url = f'{base_url}&page={page}'
        response = requests.get(url)

        if response.status_code != 200:
            print(f"Erro ao acessar o site: {response.status_code}")
            break

        soup = BeautifulSoup(response.text, 'html.parser')

        # Encontre os blocos de notícia
        for item in soup.find_all('li', class_='widget widget--card widget--info'):
            title_tag = item.find('div', class_='widget--info__title')
            link_tag = item.find('a', href=True)
            date_tag = item.find('div', class_='widget--info__meta')
            description_tag = item.find('p', class_='widget--info__description')

            if title_tag and link_tag:
                title = title_tag.get_text(strip=True)
                link = fetch_final_link(link_tag['href'])
                date = date_tag.get_text(strip=True) if date_tag else 'Desconhecida'
                description = description_tag.get_text(strip=True) if description_tag else 'Sem descrição'


                # Se a consulta não estiver no título ou descrição, pule
                if not_contains_query(query, title, description):
                    print("Pulando: ", title)
                    continue

                # Extrai o texto completo da notícia
                full_text = fetch_news_text(link)

                # Adiciona os dados ao resultado
                articles.append({
                    'title': title,
                    'link': link,
                    'date': date,
                    'description': description,
                    'full_text': full_text
                })

                # Parar se já tiver o número desejado de artigos
                if len(articles) >= num_articles:
                    break

        # Avança para a próxima página
        page += 1

    articles_df = pd.DataFrame(articles)
    articles_df = ajusta_data(articles_df)

    articles_df['candidate'] = query

    return articles_df


SyntaxError: f-string: unmatched '(' (302015075.py, line 13)

In [4]:
def fetch_final_link(link):
    """Obtém o link final de uma notícia."""
    if not link.startswith('http'):
        link = f'https:{link}'
    response = requests.get(link)
    if response.status_code != 200:
        print(f"Erro ao acessar o link da notícia: {response.status_code}")

    link_final = response.text.split('window.location.replace("')[1].split('");')[0]

    return link_final




In [10]:
def fetch_news_text(news_link):
    """Obtém o texto completo de uma notícia a partir do link."""
    if not news_link.startswith('http'):
        news_link = f'https:{news_link}'
    
    response = requests.get(news_link)
    
    if response.status_code != 200:
        print(f"Erro ao acessar o link da notícia: {response.status_code}")
        return 'Texto não disponível'
    # salvar o html
    with open('g1.html', 'w') as file:
        file.write(response.text)

    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Ajuste o seletor de acordo com a estrutura da página de notícia
    text_container = soup.find('p', class_='content-text__container')  # Exemplo de seletor
    full_text = ''
    if text_container:
        # For loop para pegar todos os p e a tag de texto
        for text in text_container:
            # concatenate the text
            full_text += text.get_text()
        return full_text
    else:
        return ''




In [38]:
def store_news(articles):
    """Armazena os dados das notícias no banco de dados PostgreSQL."""
    conn = database.connect_to_database()
    cursor = conn.cursor()

    for index, row in articles.iterrows():
        title = row['title']
        link = row['link']
        date = row['date']
        description = row['description']
        full_text = row['full_text']

        try:
            cursor.execute(
                "INSERT INTO news (title, link, date, description, full_text) VALUES (%s, %s, %s, %s, %s)",
                (title, link, date, description, full_text)
            )
        except Exception as e:
            print(f"Erro ao inserir notícia no banco de dados: {e}")

    conn.commit()
    cursor.close()
    conn.close()



In [39]:
store_news(news_articles_df)

Erro ao conectar-se ao banco de dados: could not translate host name "db" to address: Name or service not known



AttributeError: 'NoneType' object has no attribute 'cursor'

In [36]:
db_params = {
    'host': 'my_postgres',
    'database': 'mydatabase',
    'user': 'myuser',
    'password': 'mypassword',
    'port': 5432
}

# Busca notícias relacionadas a "Tabata Amaral"
query = 'Tabata Amaral'
news_articles = fetch_g1_news(query, num_articles=20)

print(f"Encontradas {len(news_articles)} notícias sobre {query}")

news_articles

2024-08-08 15:01:07.722156
2024-08-05 14:54:00
2024-08-05 21:23:00
2024-08-05 18:40:00
2024-08-05 21:38:00
2024-08-05 14:17:00
2024-08-05 15:28:00
2024-08-06 15:22:00
2024-08-06 00:13:00
2024-08-05 15:38:00
2024-08-05 15:24:00
2024-08-05 14:29:00
2024-08-05 14:22:00
2024-08-05 15:25:00
2024-08-05 13:42:00
Encontradas 15 notícias sobre Tabata Amaral


,title,link,date,description,full_text
0,Natuza Nery entrevista Tabata Amaral (PSB); ve...,https://g1.globo.com/sp/sao-paulo/eleicoes/202...,2024-08-08 15:01:07.722156,O Assunto: Natuza Nery entrevistaTabataAmaral(...,"A deputada federal Tabata Amaral, candidata à..."
1,Tabata Amaral fala sobre aborto,https://g1.globo.com/politica/eleicoes/2022/vi...,2024-08-05 14:54:00,Série do g1 começa na segunda (5) com a candid...,
2,VÍDEOS: Natuza Nery entrevista Tabata Amaral (...,https://g1.globo.com/podcast/o-assunto/playlis...,2024-08-05 21:23:00,,
3,Natuza Nery entrevista Tabata Amaral (PSB),https://g1.globo.com/podcast/o-assunto/ao-vivo...,2024-08-05 18:40:00,Natuza Nery entrevistaTabataAmaral(PSB) Esta s...,
4,O Assunto #1271: Natuza Nery entrevista Tabata...,https://g1.globo.com/podcast/o-assunto/noticia...,2024-08-05 21:38:00,... sobre traição de DatenaTabataAmaralanuncia...,"Você pode ouvir O Assunto no g1, no GloboPlay..."
5,Tabata Amaral responde sobre votos na periferia,https://g1.globo.com/politica/eleicoes/2022/vi...,2024-08-05 14:17:00,Série do g1 começa na segunda (5) com a candid...,
6,Tabata Amaral (PSB) responde pergunta sobre Sa...,https://g1.globo.com/politica/eleicoes/2022/vi...,2024-08-05 15:28:00,Série do g1 começa na segunda (5) com a candid...,
7,José Luiz Datena (PSDB) responde a pergunta so...,https://g1.globo.com/politica/eleicoes/2022/vi...,2024-08-06 15:22:00,Série do g1 começou na segunda (5) comTabataAm...,
8,Veja o que é #FATO ou #FAKE na entrevista de T...,https://g1.globo.com/fato-ou-fake/sao-paulo/no...,2024-08-06 00:13:00,"TabataAmaral(PSB), candidata à Prefeitura de S...","A deputada federal Tabata Amaral, candidata à..."
9,Tabata Amaral (PSB) responde a pergunta sobre ...,https://g1.globo.com/politica/eleicoes/2022/vi...,2024-08-05 15:38:00,Série do g1 começa na segunda (5) com a candid...,


In [29]:
news_articles_df = pd.DataFrame(news_articles)
print(news_articles_df['date'][0])
news_articles_df['date'][1]

há 5 dias


'05/08/2024 14h54'

In [22]:
news_articles_df['date'][0][3]

'5'

In [52]:
def ajusta_data(df):
    for i in range(len(df)):
        if df['date'][i][:2] == 'há':
            df['date'][i] = datetime.now() - timedelta(days=int(df['date'][i][3]))
        else:
            # convert this format 05/08/2024 14h54 to datetime
            df['date'][i] = datetime.strptime(df['date'][i], "%d/%m/%Y %Hh%M")
    return df

In [33]:
news_articles_df

,title,link,date,description,full_text
0,Natuza Nery entrevista Tabata Amaral (PSB); ve...,https://g1.globo.com/sp/sao-paulo/eleicoes/202...,2024-08-08 14:56:48.748918,O Assunto: Natuza Nery entrevistaTabataAmaral(...,"A deputada federal Tabata Amaral, candidata à..."
1,Tabata Amaral fala sobre aborto,https://g1.globo.com/politica/eleicoes/2022/vi...,2024-08-05 14:54:00,Série do g1 começa na segunda (5) com a candid...,
2,VÍDEOS: Natuza Nery entrevista Tabata Amaral (...,https://g1.globo.com/podcast/o-assunto/playlis...,2024-08-05 21:23:00,,
3,Natuza Nery entrevista Tabata Amaral (PSB),https://g1.globo.com/podcast/o-assunto/ao-vivo...,2024-08-05 18:40:00,Natuza Nery entrevistaTabataAmaral(PSB) Esta s...,
4,O Assunto #1271: Natuza Nery entrevista Tabata...,https://g1.globo.com/podcast/o-assunto/noticia...,2024-08-05 21:38:00,... sobre traição de DatenaTabataAmaralanuncia...,"Você pode ouvir O Assunto no g1, no GloboPlay..."
5,Tabata Amaral responde sobre votos na periferia,https://g1.globo.com/politica/eleicoes/2022/vi...,2024-08-05 14:17:00,Série do g1 começa na segunda (5) com a candid...,
6,Tabata Amaral (PSB) responde pergunta sobre Sa...,https://g1.globo.com/politica/eleicoes/2022/vi...,2024-08-05 15:28:00,Série do g1 começa na segunda (5) com a candid...,
7,José Luiz Datena (PSDB) responde a pergunta so...,https://g1.globo.com/politica/eleicoes/2022/vi...,2024-08-06 15:22:00,Série do g1 começou na segunda (5) comTabataAm...,
8,Veja o que é #FATO ou #FAKE na entrevista de T...,https://g1.globo.com/fato-ou-fake/sao-paulo/no...,2024-08-06 00:13:00,"TabataAmaral(PSB), candidata à Prefeitura de S...","A deputada federal Tabata Amaral, candidata à..."
9,Tabata Amaral (PSB) responde a pergunta sobre ...,https://g1.globo.com/politica/eleicoes/2022/vi...,2024-08-05 15:38:00,Série do g1 começa na segunda (5) com a candid...,


In [55]:
articles = fetch_g1_news('Guilherme Boulos', num_articles=20)
articles

,title,link,date,description,full_text,candidate
0,O Assunto #1273: Natuza Nery entrevista Guilhe...,https://g1.globo.com/podcast/o-assunto/noticia...,2024-08-08 19:55:22.022866,"...GuilhermeBoulos(PSOL) em entrevista ao g1, ...","Você pode ouvir O Assunto no g1, no GloboPlay...",Guilherme+Boulos
1,Natuza Nery entrevista Guilherme Boulos (PSOL)...,https://g1.globo.com/sp/sao-paulo/eleicoes/202...,2024-08-08 19:55:22.023088,O Assunto: Natuza Nery entrevistaGuilhermeBoul...,"Guilherme Boulos, candidato à Prefeitura de S...",Guilherme+Boulos
2,Veja o que é #FATO ou #FAKE na entrevista de G...,https://g1.globo.com/fato-ou-fake/sao-paulo/no...,2024-08-08 19:55:22.023174,"GuilhermeBoulos(PSOL) em entrevista ao g1, nes...","O deputado federal Guilherme Boulos, candidat...",Guilherme+Boulos
3,Datafolha: veja intenção de voto em possível 2...,https://g1.globo.com/sp/sao-paulo/eleicoes/202...,2024-08-06 19:55:22.023241,"O prefeito de São Paulo, Ricardo Nunes (MDB), ...",Pesquisa Datafolha divulgada nesta terça-feir...,Guilherme+Boulos
4,"Datafolha: Boulos (35%), Datena (31%) e Marçal...",https://g1.globo.com/sp/sao-paulo/eleicoes/202...,2024-08-09 19:55:22.023303,"...-feira (8) aponta queGuilhermeBoulos(PSOL),...",Pesquisa Datafolha desta quinta (8) mostra qu...,Guilherme+Boulos
5,Justiça ordena pela 2ª vez que Marçal apague i...,https://g1.globo.com/sp/sao-paulo/eleicoes/202...,2024-08-06 19:55:22.023364,Os candidatos à Prefeitura de SP Pablo Marçal ...,O Tribunal Regional Eleitoral de SP (TRE-SP) ...,Guilherme+Boulos
6,Eleições 2024 em São Paulo: Ricardo Nunes tem ...,https://g1.globo.com/sp/sao-paulo/eleicoes/202...,2024-07-05 20:10:00,".... Na sequência, estãoGuilhermeBoulos(PSOL),...",Pesquisa Quaest divulgada nesta quinta-feira ...,Guilherme+Boulos
7,Eleições 2024: confira raio-X dos principais p...,https://g1.globo.com/sp/sao-paulo/noticia/2024...,2024-08-08 19:55:22.023600,... dos primeiros colocados segundo a última p...,Apesar de convenções partidárias - que aconte...,Guilherme+Boulos
8,Resumão diário #1048: Ex-executivos das Lojas ...,https://g1.globo.com/podcast/resumao-diario/no...,2024-06-27 13:19:00,... bilhões. Eleições 2024 em São Paulo: Ricar...,"Você pode ouvir o Resumão Diário no g1, no Gl...",Guilherme+Boulos
9,O Assunto entrevista João Pimenta,https://g1.globo.com/podcast/o-assunto/noticia...,2024-08-12 19:55:22.023741,"... e não apoiarGuilhermeBoulos(PSOL), João di...","Você pode ouvir O Assunto no g1, no GloboPlay...",Guilherme+Boulos
